# Hackathon 2

In the first exercise, you will develop a model for estimating the cooling/heating load indexes of a building from its characteristics. These indexes serve as reference to adapt the power of air conditioning systems. In the second exercise, you will forecast the hourly electric power consumption in Spain. Electricity being not storable, predicting the electric consumption is a crucial matter for energy producers. You will develop an autoregressive model for this purpose. 

## Report content

•	You have to fill in this  jupyter notebook downloadable on the moodle website of the course

•	Grades are granted to the members whose names are in the Jupyter notebook. If your name doesn’t appear on the top of the notebook, you’ll get a 0, even though you are in a group on Moodle.

•	The jupyter notebook must be compiled with printed results and next submitted via moodle. The absence of compiled results (or non-printed values) leads to a lower grade.

## Report submission

•	The deadline for submission is reported on the moodle website. Submission after the deadline will not be accepted.

•	To submit your report, go to the section “APP” on Moodle and the subsection “Soumission du rapport”. You can upload your work there. Once you are sure that it is your final version, click the button “Envoyer le devoir”. It is important that you don’t forget to click on this button ! 

•	Reports that have not been uploaded through Moodle will not be corrected.

## Names and Noma of participants:

Part. 1: Martin Gyselinck 19282000

Part. 2: Ysaline Paque 18802000

Part. 3: Isaline Deckers 21172000

Part. 4: Camille D'Hont 21012000

Part. 5: Jean de Briey 37941700

Part. 6: Guillaume Spronck 48131900

## Regression

When designing a building, the computation of the heating load (HL) and the cooling load (CL) is required to determine the specifications of the heating and cooling equipment needed to maintain comfortable indoor air conditions. Architects and building designers need information about the characteristics of the building and of the conditioned space. For this reason, we  investigate the effect of eight input variables: (RC), surface area, wall area, roof area, overall height, orientation, glazing area, and glazing area distribution, to determine the output variables HL and CL of residential buildings.
The dataset contains eight attributes (or features) and two responses (or outcomes). The aim is to use the eight features to predict each of the two responses.

Features: RelativeCompactness, SurfaceArea ,WallArea, RoofArea, OverallHeight, Orientation, GlazingArea, GlazingAreaDistribution.

Prediction: HeatingLoad, CoolingLoad



---
1) Report useful statistics for each variables (mean, std, heatmap of correlations,...)

In [ ]:
import csv
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
from datetime import datetime

data = pd.read_csv("Data_heating_cooling.csv")
df = pd.DataFrame(data)

variables = list(df.columns)
mdf = df.mean()
mediandf = df.median()
stddf = df.std()
quantile_5df = df.quantile(0.05)
quantile_95df = df.quantile(0.95)

print(df.describe(percentiles=[.05, .95]).round(decimals=3))

fig, axes = plt.subplots(nrows=3, ncols=2, figsize=(11, 10))  # create 4x2 array of subplots

df.boxplot(column='Relative_Compactness', ax=axes[0, 0])
df.boxplot(column='Surface_Area', ax=axes[0, 1])
df.boxplot(column='Wall_Area', ax=axes[1, 0])
df.boxplot(column='Roof_Area', ax=axes[1, 1])
df.boxplot(column='Overall_Height', ax=axes[2, 0])
df.boxplot(column='Glazing_Area', ax=axes[2,1])
fig.suptitle(' Boxplot for each feature ', fontsize=20)
plt.show()
plt.title("Correlation Matrix of the differents features")
sns.heatmap(df.drop(columns=['Heating_Load', 'Cooling_Load']).corr().round(decimals=3), annot=True, cmap='Greens', xticklabels=True)
df = pd.get_dummies(df, columns=['Orientation', "Glazing_Area_Distribution"], prefix=['Orientation', "Glazing_Area_Distribution"], drop_first=True)



For this question we simply extracted the data, dropped the columns HeatingLoad and CoolingLoad since we want to analyze the features. Then we used the describe function available for the dataframe which gives us the parameters for each feature. We then made a boxplot for each numerical variable and finally calculated and printed the correlation between each feature. Finally, we transformed the categorical variables(Orientation and Glazing Area Distribution) using the get Dummies function.

We can see that the features: Relative Compactness, Area, Roof Area and Overall Height are strongly correlated to each other and Wall Area is weakly correlated to these features. Orientation is completely decorrelated from all other features. Glazing Area and Glazing Area Distribution are weakly correlated to each other and completely decorrelated to the other features. Note that Orientation and Glazing Area Distribution are dummies variable.

---
2)	Add a constant term (intercept) and perform a linear regression of the variable CoolingLoad on all other explanatory variables. Use  the library statsmodels.

i. Report the F statistics and R2: interpret them. 

ii. Analyze the t-statistics and p-values of each coefficient of regression.

Are all coefficients significant at 95%? Use the library statsmodels.api. The function OLS accepts pandas dataframe (use .drop() to remove columns).


In [ ]:
y = df['Cooling_Load']

x = sm.add_constant(df.drop(columns=['Cooling_Load', 'Heating_Load', 'Roof_Area'], axis=1))

model = sm.OLS(y, x).fit()
predictions = model.predict(x)

print(model.summary())

To perform a linear regression on the predictive variable CoolingLoad we first dropped the variable Roof Area since it is a linear combination of Surface Area and Wall Area. We then added a constant to all the other variables before computing the linear regression.

(i) F-statistic = 471.3 and R-squared = 0.89 which means that the independant variables in the model are significantly related to y (CoolingLoad). In addition we can see that the p-value ('Prob (F-statistic)' -- which is the probability that the null hypothesis for the full model is true) is zero. Since the p-value is zero we reject the null hypothesis, this means, in oder words, that it is obvious that there is a linear relationship. 

(ii) All coefficients are significant at the 95% level, except for the variables 'Orientation_3', 'Orientation_4' and 'Orientation_5' whose p-value is higher than 5%.

---
3)	Remove non-significant coefficients except the intercept (e.g. with  𝛼  5%) and run again the regression. What do you notice when you compare the R2, log-likelihood, AIC and BIC (the AIC and BIC are not explained in the course, search on internet for explanations)


In [ ]:
x = df.drop(columns=['Cooling_Load', 'Heating_Load', 'Roof_Area', "Orientation_3", "Orientation_4", "Orientation_5"], axis=1)
y = df['Cooling_Load']

x = sm.add_constant(x)

model = sm.OLS(y, x).fit()

print(model.summary())

After droping 'Orientation_3', 'Orientation_4' and 'Orientation_5' since they are not significant and perform again the linear regression, we can observe that R-squared hasn't change (0.89) which means the model is not better or worse that before. It confirm that the three variables we dropped were unsignificant. The Log-Likelihood is slightly worse (-2.1 from before) but it is normal since we removed three variables. Deleting some predictors variables in a model will always lower the log-likelihood value, even if the deleted predictors variables were not statistically significant. 

The AIC can be viewed as a measure of the goodness of fit of any estimated statistical model. as the is a type of model selection from a class of parametric models with different numbers of parameters. The values of AIC and BIC attempt to resolve the problem of overfitting. An overfitted model has more parameters that can be justified by the data. Since AIC and BIC are estimator of prefiction error, a lower AIC or BIC value indicates a better fit. In comparison with our first regression, the AIC decreased by 2 and the BIC by 16, which means that the second model is a better fit.

## Auto-regressive model

In a paper released early 2019, forecasting in energy markets is identified as one of the highest leverage contribution areas of Machine/Deep Learning toward transitioning to a renewable based electrical infrastructure (see https://arxiv.org/abs/1906.05433).  The file “Data_energy_load.csv” contains 4 years of electrical consumption  for Spain in MW/h.  This was retrieved from ENTSOE a public portal for Transmission Service Operator (TSO) data.

File format:

Date: dd-mm-yy, Hour : from 0 to 23. Weekday: from 1 (Sunday)  to 7 (Saturday). Load: consumption in MW/h. Lm1: consumption 1h ago, Lm2: consumption 2h ago,… Lm14: consumption 14h ago

1)	Load the dataset and convert dates in datetime format (you can use the package datetime). Plot the time series of consumption, what do you observe?

In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as sc
import datetime as dt
import matplotlib.pyplot as plt

data_energy_load = pd.read_csv("Data_energy_load.csv", sep=";", skipinitialspace=True)

df = pd.DataFrame(data_energy_load)
df["Datetime"] = pd.to_datetime(df["Date"].astype(str) + "/" + df["Hour"].astype(str) + "/" + (df["Weekday"] - 1).astype(str),
                                format="%d-%m-%y/%H/%w")

plt.figure(figsize=(15, 5))
plt.plot(df["Datetime"].to_numpy(), df["Load"].to_numpy())
plt.xlabel('Date')
plt.ylabel('Load [MW/h]')
plt.show()

We can see that all the loads oscillate widely between 20000 and 40000 MW/h. We can also see that the values do not change significantly according to the different periods of the year. 

---
2) We will regress the electric consumption (‘Load’) on the following covariates ‘Weekday’, ‘Hour’, ‘Lm1’, ‘Lm2’,…,’Lm14’. Plot the following graphs:

•	Average electric consumption per day (y axis) versus weekday (x axis)

•	Average electric consumption per hour(y axis) versus hour (x axis)

Based on these graphs, do you think that it is a good idea to regress linearly the consumption on variables ‘Hour’ and ‘Weekday’? 


In [ ]:
plt.grid(True)
listday = range(1, 8)
loadavperday = []
for i in listday:
    loadavperday.append(np.mean(df[df.get('Weekday') == i].get('Load')))
plt.scatter(listday, np.multiply(loadavperday, 24))  #moyenne
plt.title("Average electric consumption per day versus weekday")
plt.xlabel("Weekday")
plt.ylabel("Consumption per day [MW/day]")
plt.show()

plt.grid(True)
listhour = range(24)
loadavperhour = []
for i in listhour:
    loadavperhour.append(np.mean(df[df.get('Hour') == i].get('Load')))
plt.scatter(listhour, loadavperhour)  #moyenne
plt.title("Average electric consumption per hour versus hour")
plt.xlabel("Hour [h]")
plt.ylabel("Consumption per hour [MW/h]")
plt.show()


Our first graph represents the average electricity consumption according to the day of the week and the second graph represents the average electricity consumption according the hour of the day.

We observe that it is not a good idea to regress linearly on the variables 'Hour' and 'Weekday' for either case. Indeed, the average consumptions vary very strongly according to the hours of the day and according to the days of the week. If we draw a straight line to estimate the average consumption values, we would have very large errors between the estimate and the real values.

---
3) We will slightly transform the dataset. We first add a constant column for the regression (column of ones). You also convert the variable ‘Hour’ into 23 (and not 24!) binary variables H1,…H23. You next convert the variable ‘Weekday’ into 6 (and not 7) variables W2,…,W7. Use the command get_dummies(.) from pandas. Why do we remove H0 and W1? Why do we do this conversion?

---

In [ ]:
df.insert(len(df.columns) - 1, "regression", 1)
df = pd.get_dummies(df, columns=["Hour", "Weekday"], prefix=["Hour", "Day"], drop_first=True)

With the data of hours and weekdays we create two matrices. For each off those matrices we have to remove the first column (H0 and W1). By removing the first column we transform linearly dependent matrices into linearly independent matrices. To resolve this exercise we need matrices that are linearly independant.

---
4) Split the dataset in

•	a training set with observations from 02-01-15 up to 01-12-2018 (included)

•	a test (or validation) set with observations from 02-12-2018 up to 31-12-2018.


In [ ]:
training_set = df[(datetime(2015, 1, 2) <= df["Datetime"]) & (datetime(2018, 12, 1, 23) >= df["Datetime"])]
test_set = df[(datetime(2018, 12, 2) <= df["Datetime"]) & (datetime(2018, 12, 31, 23) >= df["Datetime"])]

---
If we denote by L(t) the consumption at time t, the model that we want to develop is 
$$L(t) = cst + \sum_{k=1}^{14} a_k L(t-k) +\sum_{k=2}^{7} b_k W_k + \sum_{k=1}^{23} c_k H_k    $$

o	Estimate this model with statsmodels on the training set. 

o	How would you judge the quality of the predictive model?

o	Compute the Mean Absolute Error (MAE) between predicted and real consumptions.



In [ ]:
X = training_set.drop(["Datetime", "Load", "Date"], axis=1)
Y = training_set["Load"]

model = sm.OLS(Y, X).fit()

predictions1 = model.predict(X)


def computeMAE(Y, y):
    sum = 0
    for i in range(len(y)):
        sum += abs(Y[i] - y[i])
    return sum / len(y)


MAE = computeMAE(Y, predictions1)
print(model.summary())
print("R2 = ", model.rsquared)
print('MAE = ', MAE)

We can judge the quality of a predictive model with the R-squared. In our case, it is worth 97.64% which is really good.

The MAE on the trainging set is the average error that the model predictions have with respect to their corresponding true values. For our model, the average error between the prediction and the reality is 444.52 MW/h, while the consumptions oscillate between 20 and 30 kMW/h. We can therefore say that we obtain a good value of MAE since the values of our dataset are much higher then the values of the MAE.

5) Use this model on the test set to forecast the electric consumption.

o	Compare on a graph, the forecast to  real consumptions on the given period. Plot also the errors of prediction.

o	Compute the MAE on the test set and the R2. Is the forecast reliable?

In [ ]:
X = test_set.drop(["Datetime", "Load", "Date"], axis=1)
predictions2 = model.predict(X)
real = test_set["Load"]
x = np.arange(len(predictions2))

plt.figure(figsize=(15, 5))
plt.plot(x, predictions2, label='predicted consumptions')
plt.plot(x, real, label='real consumptions')
plt.xlabel('time')
plt.ylabel('load [MW/h]')
plt.legend()
plt.show()
plt.figure(figsize=(15, 5))
plt.scatter(x, abs(real-predictions2), label="error", s=4)
plt.xlabel('time')
plt.ylabel('load [MW/h]')
plt.legend()
plt.show()
MAE2 = computeMAE(real.to_numpy(), predictions2.to_numpy())
SSR = np.sum((predictions2 - np.mean(test_set["Load"].to_numpy())) ** 2)
SST = np.sum((test_set["Load"] - np.mean(test_set["Load"].to_numpy())) ** 2)
print(f"R2 = {SSR / SST:0.04f}")
print(f'MAE = {MAE2:0.04f}', )


The R2 on the test set is 97.04%, it is a little smaller than the one on the training set. But,as for question 4, we can say that it is very good and that the predictions are reliable. 
The MAE on the test set is 444.82 MW/h, which is still minimal compared to the values on our test set. So we have a very good MAE too.

We can conclude that our model estimates very well both a sample over several years and a sample over one month.

6) We want to check that the average consumption at 11h is significantly different (at 95%) from the average consumption at 13h. Perform a statistical test (explain which stat you use) and report the stat value, the pivot value(s) and the p-value.

In [ ]:
alpha = 0.05
data_energy_load = pd.read_csv("Data_energy_load.csv", sep=";", skipinitialspace=True)

df = pd.DataFrame(data_energy_load)
H11 = df[df.get('Hour') == 11].get('Load')
H13 = df[df.get('Hour') == 13].get('Load')

M11 = np.mean(H11)
M13 = np.mean(H13)

STD11 = np.std(H11, ddof=1)
STD13 = np.std(H13, ddof=1)

LEN11 = len(H11)
LEN13 = len(H13)

PoolSTD = np.sqrt(((LEN11 - 1) * STD11 * STD11 + (LEN13 - 1) * STD13 * STD13) / (LEN11 + LEN13 - 2))

SEM = PoolSTD * np.sqrt((1 / LEN11) + (1 / LEN13))
DeFr = LEN11 + LEN13 - 2
T = (M11 - M13)

t = (M11 - M13) / SEM
pval = 2 * sc.t.cdf(-np.abs(t), df=DeFr)
t_l, t_u = sc.t.ppf(q=alpha / 2, df=DeFr), sc.t.ppf(q=1 - alpha / 2, df=DeFr)

print("T-test : ")
print("H0 : µ11 = µ13")
print("H1 : µ11 ≠ µ13", end="\n\n")

print(f"The t value : {t:0.04f}")
print(f"Pivots : t_l = {t_l:0.4f} t_u = {t_u:0.4f}")
print(f"P-value = {pval:0.04f} < 0.05 so we reject the null hypothesis")

Because we are testing 2 sample means and the σ are unknown, we use this test: $T(X_{11}, X_{13})=\frac{(\overline{X}_{11}-\overline{X}_{13})-0}{S_{pool}*\sqrt{\frac{1}{n_{11}}+\frac{1}{n_{13}}}}$ with hypotheses $H_{0} : µ_{11} = µ_{13}$ and $H_{1} : µ_{11} ≠ µ_{13}$.

We reject $H0$  at the level 5% if:

•	we find $T(X_{11}, X_{13})<t_{n_{11}+n_{13}-2~~0.05/2}$ or if $T(X_{11}, X_{13})>t_{n_{11}+n_{13}-2~~1-0.05/2}$, where "$t_{n_{11}+n_{13}-2~~α}$" is the α-percentile of a Student's T.

•	And/or we find a p-value smaller than 0,05.

We can easily confirm with the results we have that the null hypothesis is rejected. The t value that we we found (-2.1686) is indeed outside the range [t_l t_u] we found(-1.9608 and 1.9608). This is confirmed by the p-value (0.0302) which is smaller than 0.05.

7) The test of question 6 requires that variances are equal. Test if this assumption holds (95% confidence level). Report  the stat value, the pivot value (s).

In [ ]:
Tobs = STD11 ** 2 / STD13 ** 2
fmax = sc.f.ppf(q=0.975, dfn=LEN11 - 1, dfd=LEN13 - 1)
fmin = sc.f.ppf(q=0.025, dfn=LEN11 - 1, dfd=LEN13 - 1)
pval = 2 * min(sc.f.cdf(Tobs, dfn = LEN11-1, dfd = LEN13-1), 1 - sc.f.cdf(Tobs, dfn = LEN11-1, dfd = LEN13-1))
print(f"Pivots : {fmin:0.4f} {fmax:0.4f}")
print(f"Tobs : {Tobs:0.4f}")
print(f"P-value : {pval:0.04f}")

Because we are testing on 2 variances, we use the test $T(X_{11}, X_{13})=\frac{{S}_{11}^{2}}{{S}_{13}^{2}}$ with hypotheses $H_{0} : σ_{11}^{2} = σ_{13}^{2}$ and $H_{1} : σ_{11} ≠ σ_{13}$.

We reject $H0$  at the level 5% if we find $T(X_{11}, X_{13})<F_{n_{11}+n_{13}-2~~0.05/2}$ or if $T(X_{11}, X_{13})>F_{n_{11}+n_{13}-2~~1-0.05/2}$

So we do not reject $H_{0}$ because we found fmin < Tobs < fmax (fmin and fmax are the pivots of our test).

We reach the same conclusion with the p-value. Indeed, our p-value is greater than 5%.